In [3]:
import pandas as pd
import numpy as np
import requests
import datetime
import time

In [4]:
bdata = pd.read_html("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20150101&end="+time.strftime("%Y%m%d"))[0]

In [5]:
bdata = bdata.assign(Date=pd.to_datetime(bdata['Date']))
bdata['Volume'] = bdata['Volume'].astype('int64')

kwargs = {
            'Close Off High': lambda x: 2*(x['High'] - x['Close'])/(x['High'] - x['Low'])-1,
             'Volatility': lambda x: (x['High'] - x['Low'])/(x['Open'])
         }

bdata = bdata.assign(**kwargs)

# Get only relevant columns
mdata = bdata[['Date']+['Close']+['Volume']+['Close Off High']+['Volatility']]
mdata = mdata.sort_values(by='Date')

In [6]:
mdata.head()

,Date,Close,Volume,Close Off High,Volatility
1205,2015-01-01,314.25,8036550,0.922360,0.020097
1204,2015-01-02,315.03,7860650,-0.289474,0.007259
1203,2015-01-03,281.08,33054400,1.000000,0.108210
1202,2015-01-04,264.19,55629100,0.555706,0.105353
1201,2015-01-05,274.47,43962800,-0.416290,0.050023


In [21]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

In [22]:
def train_test_rmse(df, feature_cols, testsize):
    X = df[feature_cols]
    y = df['Close']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=80, test_size=testsize)
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)

    y_pred = linreg.predict(X_test)
    
    plt.scatter(y_test, y_pred)
    plt.xlabel('Actual Prices')
    plt.ylabel('Predicted Prices')
    print('RMSE: %f'%np.sqrt(mse(y_test, y_pred)))

In [24]:
mdata.shape

(1206, 5)

In [25]:
from swifter import swiftapply